In [40]:
import requests
from urllib.parse import quote_plus

import pandas

In [14]:
import bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, BooleanFilter, CDSView, tools

In [3]:
output_notebook()

Loading BokehJS ...

In [76]:
z_min = 1.
z_max = 2.
lmass_max = 10.
url_search = 'http://z.mast.stsci.edu/api/v0.1/candels?catalog=candels_master_view&zphot.gt={}&zphot.lt={}&lmass.gt={}'.format(z_min, z_max, lmass_max)
r = requests.get(url_search)

In [77]:
colnames = []
for col in r.json()['info']:
    colnames.append(col['name'])

json_data = r.json()['data']
df = pandas.DataFrame(json_data, columns = colnames)

In [78]:
df

,objID,ID,IAU_Name,RA,DEC,Field,PhotFlag,Class_star,AGNFlag,zbest,...,WFC3_F160M_MAG,IRAC_CH1_MAG,IRAC_CH2_MAG,IRAC_CH3_MAG,IRAC_CH4_MAG,A_IMAGE,B_IMAGE,KRON_RADIUS,FWHM_IMAGE_F160W,FLUX_RADIUS_2_F160W
0,1000000032,32,CANDELS_GDS_F160W_J033219.72-275713.1,53.082191,-27.953646,GOODS-S,0.0,0.01,0.0,2.5640,...,24.527438,22.575933,22.618292,24.034591,23.059972,1.952,0.799,9.02,7.460000,4.488
1,1000000040,40,CANDELS_GDS_F160W_J033221.18-275715.3,53.088264,-27.954256,GOODS-S,0.0,0.02,0.0,1.0860,...,21.436636,20.554003,20.780140,21.196497,21.190223,7.522,5.960,3.57,10.090000,7.242
2,1000000076,76,CANDELS_GDS_F160W_J033219.97-275704.7,53.083210,-27.951328,GOODS-S,0.0,0.02,0.0,1.1490,...,20.992402,19.648451,19.775395,20.118456,19.983871,11.286,6.592,3.88,15.520000,9.849
3,1000000079,79,CANDELS_GDS_F160W_J033221.72-275701.9,53.090523,-27.950546,GOODS-S,0.0,0.00,0.0,1.0400,...,20.488726,19.323211,19.509646,19.803627,19.826733,10.436,7.621,3.61,25.530001,10.860
4,1000000080,80,CANDELS_GDS_F160W_J033218.63-275658.7,53.077648,-27.949656,GOODS-S,0.0,0.00,0.0,1.1050,...,21.830635,20.971884,21.197695,21.618959,21.669452,6.933,4.634,3.50,18.100000,7.402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4546,1400027423,27423,CANDELS_GDN_J123640.5+621833.1,189.168911,62.309187,GOODS-N,NaN,0.00,NaN,1.1456,...,20.418700,19.364300,19.438100,19.878200,20.054300,16.381,7.599,3.50,12.090000,8.620
4547,1400027424,27424,CANDELS_GDN_J123623.7+621837.2,189.098551,62.310330,GOODS-N,NaN,0.00,NaN,1.0126,...,20.306100,19.300200,19.510200,NaN,19.874000,11.728,6.400,5.28,6.670000,8.604
4548,1400027438,27438,CANDELS_GDN_J123637.5+621832.8,189.156344,62.309101,GOODS-N,NaN,0.00,NaN,1.5570,...,23.063001,22.062401,21.753101,21.713500,22.156700,5.686,4.208,4.62,5.230000,4.489
4549,1400030979,30979,CANDELS_GDN_J123747.1+621334.1,189.446089,62.226134,GOODS-N,NaN,0.00,NaN,1.9890,...,24.781494,22.858798,23.106801,24.343501,23.436400,2.619,0.841,10.56,8.880000,6.956


In [79]:
print(colnames)

['objID', 'ID', 'IAU_Name', 'RA', 'DEC', 'Field', 'PhotFlag', 'Class_star', 'AGNFlag', 'zbest', 'zspec', 'zphot', 'lmass', 'lmass_std', 'ACS_F435W_MAG', 'ACS_F606W_MAG', 'ACS_F775W_MAG', 'ACS_F814W_MAG', 'ACS_F850LP_MAG', 'WFC3_F098M_MAG', 'WFC3_F105M_MAG', 'WFC3_F125M_MAG', 'WFC3_F140W_MAG', 'WFC3_F160M_MAG', 'IRAC_CH1_MAG', 'IRAC_CH2_MAG', 'IRAC_CH3_MAG', 'IRAC_CH4_MAG', 'A_IMAGE', 'B_IMAGE', 'KRON_RADIUS', 'FWHM_IMAGE_F160W', 'FLUX_RADIUS_2_F160W']


In [80]:
gal = (df['Class_star'] < 0.1) & (df['PhotFlag']==0)

source = ColumnDataSource(df)
view = CDSView(source=source, filters=[BooleanFilter(gal)])

p = figure(
    toolbar_location="above",
    title='Mass-Redshift Plot',
    x_range=(0.,5.1),
    y_range=(9.75,13.5))

p.xaxis.axis_label = r'z_{phot}'
p.yaxis.axis_label = r'log_{10}\,M_{\odot}'

hover = tools.HoverTool()
hover.tooltips=[("ID", "@IAU_Name")]
p.add_tools(hover)



p.circle('zbest', 'lmass', size=5, source=source, color='gray', hover_color="red", alpha=0.1)
p.circle('zbest', 'lmass', size=5, source=source, view=view, color='lightskyblue', hover_color="red")

show(p)

In [81]:
df['PhotFlag'].describe()

count    3675.000000
mean        0.100136
std         0.458142
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         6.000000
Name: PhotFlag, dtype: float64

In [ ]:
## Need to loop trough catalog to get other data!
ra, dec
url_search = F'http://z.mast.stsci.edu/api/v0.1/candels?catalog=candels_photometry_view&ra={df["RA"][0]}&dec={df["DEC"][0]}&radius=0.0001'
r = requests.get(url_search)
colnames = []
for col in r.json()['info']:
    colnames.append(col['name'])

json_data = r.json()['data']
df_id = pandas.DataFrame(json_data, columns = colnames)

for index, row in df[1:].iterrows():
    if row['IAU_Name'].startswith('CANDELS_GDS'):
        url_search = F'http://z.mast.stsci.edu/api/v0.1/candels?catalog=candels_photometry_view&ra={row["RA"]}&dec={row["DEC"]}&radius=0.0001'
        r = requests.get(url_search)
        json_data = r.json()['data']
        df_id = df_id.append(json_data, ignore_index=True)

In [ ]:
df_id

In [ ]:
len(colnames)